In [1]:
from datasets import load_dataset

In [44]:
# Load the datasets
# ds1_train = load_dataset("joelniklaus/legal_case_document_summarization", split='train')
# ds1_train = ds1_train.remove_columns(['dataset_name'])
# ds1_train = ds1_train.rename_column('judgement', 'text')
# ds1_train = ds1_train.rename_column('summary', 'labels')
# print(ds1_train)

# ds1_test = load_dataset("joelniklaus/legal_case_document_summarization", split='test')
# ds1_test = ds1_test.remove_columns(['dataset_name'])
# ds1_test = ds1_test.rename_column('judgement', 'text')
# ds1_test = ds1_test.rename_column('summary', 'labels')

# # NOTE: This dataset only has 50 rows. It may not be a dataset we want to use.
# # NOTE: THIS DATA IS NOT PLAYING NICELY WITH CONCATENATION
# # Although the summaries appear to be good
# ds2 = load_dataset("manasvikalyan/legal-documents-summary")
# ds2 = ds2['data']
# ds2 = ds2.remove_columns(['summary_a2'])
# ds2 = ds2.rename_column('summary_a1', 'labels')
# ds2 = ds2.rename_column('judgement', 'text')
# print(ds2)

In [3]:
# ds9: AjayMukundS/Legal_Text_Summarization-llama2
ds9_train = load_dataset("AjayMukundS/Legal_Text_Summarization-llama2", split='train')
ds9_test = load_dataset("AjayMukundS/Legal_Text_Summarization-llama2", split='test')
print(ds9_train)

Dataset({
    features: ['judgement', 'dataset_name', 'summary', 'text'],
    num_rows: 7773
})


In [4]:
from transformers import BartTokenizer

In [ ]:
# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

In [26]:
from transformers import AutoTokenizer

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [30]:
# Tokenize the input texts
def preprocess_data(batch):
    inputs = batch['text']
    targets = batch['summary']
    
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [31]:
# Tokenize the datasets for DistilBART
# Apply preprocessing to the dataset
train_dataset = ds9_train.map(preprocess_data, batched=True, remove_columns=ds9_train.column_names)
test_dataset = ds9_test.map(preprocess_data, batched=True, remove_columns=ds9_test.column_names)

Map:   0%|          | 0/7773 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [34]:
# Set dataset format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [35]:
# Taking 10 examples from each tokenized set to test the training.
train_sample = train_dataset.select(range(30))
test_sample = test_dataset.select(range(10))

print(train_sample)
print(test_sample)


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 30
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})


In [11]:
from datasets import concatenate_datasets

In [12]:
# Maybe do later?

In [36]:
from transformers import BartForConditionalGeneration

In [ ]:
# Load the DistilBART model for conditional generation
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')

In [15]:
# To handle padding dynamically (i.e., pad to the longest sequence in a batch rather than a fixed length)
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
from transformers import TrainingArguments, Trainer, logging

In [17]:
import torch
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device
print(f"Accelerator is using device: {device}")

Accelerator is using device: cpu


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
)

In [39]:
# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_sample,
    eval_dataset=test_sample,
    tokenizer=tokenizer
)

In [40]:
# Train the model
trainer.train()

***** Running training *****
  Num examples = 30
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 305,510,400


Epoch,Training Loss,Validation Loss
1,No log,2.389033
2,No log,2.340875
3,No log,2.342005



***** Running Evaluation *****
  Num examples = 10
  Batch size = 4

***** Running Evaluation *****
  Num examples = 10
  Batch size = 4
Saving model checkpoint to ./results\checkpoint-24
C:\Users\Sterling\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Configuration saved in ./results\checkpoint-24\config.json
Configuration saved in ./results\checkpoint-24\generation_config.json
Model weights saved in ./results\checkpoint-24\model.safetensors
tokenizer config file saved in ./results\checkpoint-24\tokenizer_config.json
Special tokens file saved in ./results\c

TrainOutput(global_step=24, training_loss=2.1253064473470054, metrics={'train_runtime': 343.4027, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.07, 'total_flos': 139312087695360.0, 'train_loss': 2.1253064473470054, 'epoch': 3.0})

In [21]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, text, judgement, dataset_name. If summary, text, judgement, dataset_name are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 10
  Batch size = 4


{'eval_loss': 2.6838903427124023, 'eval_runtime': 11.2525, 'eval_samples_per_second': 0.889, 'eval_steps_per_second': 0.267, 'epoch': 3.0}


In [ ]:
import random

# Get a random index
random_index = random.randint(0, len(ds9_test_tokenized) - 1)

# Access the 'text' property of the randomly selected sample
random_sample_text = ds9_test_tokenized[random_index]['text']
print(random_sample_text)

In [43]:
# Example of generating a summary

inputs = [random_sample_text]

# Tokenize the input
input_ids = tokenizer(inputs, return_tensors="pt", max_length=1024, truncation=True, padding="max_length").input_ids

# Generate summary
outputs = model.generate(input_ids=input_ids, max_length=128, num_beams=4, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(summary)



The question at issue on this appeal is whether two provisions of the State Immunity Act 1978 are consistent with the European Convention on Human Rights and the European Union Charter of Fundamental Rights.
The EAT declared that sections 4(2)(b) and 16(1)(a) of the Act should be disapplied so far as they prevented Ms Janah from bringing claims based on EU law, on the ground that they were contrary to the right of access to a court in order to make a declaration of incompatibility under section 4 of the Human Rights Act 1998.
By the time of her dismissal she was permanently resident


In [ ]:
# Save your model
model.save_pretrained('./trained_HX100_model_2')

# Save the tokenizer
tokenizer.save_pretrained('./trained_HX100_model_2')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('./trained_HX100_model_2')
tokenizer = AutoTokenizer.from_pretrained('./trained_HX100_model_2')

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [45]:
import random

# Select 5 random indices from the combined_testing_tokenized_dataset
random_indices = random.sample(range(len(ds9_test_tokenized)), 5)

for idx in random_indices:
    # Example: Take the tokenized input from the tokenized testing dataset
    example_input_ids = ds9_test_tokenized['input_ids'][idx]
    
    # Convert to torch tensor and move to the device (GPU or CPU)
    example_input_ids = torch.tensor(example_input_ids).unsqueeze(0).to(device)
    
    # Generate the summary using the model
    summary_ids = model.generate(example_input_ids, num_beams=4, max_length=200, early_stopping=True)
    
    # Decode the generated summary
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Decode the actual (original) input summary
    original_summary = tokenizer.decode(example_input_ids[0], skip_special_tokens=True)
    
    # Print both summaries for comparison
    print("------------------------------------------------------------------------------------------")
    print(f"Example {idx + 1}:")
    print("Original Summary:", original_summary)
    print("*****************************************************************************************")
    print("Generated Summary:", generated_summary)
    print("\n")


------------------------------------------------------------------------------------------
Example 29:
Original Summary: [INST] Summarize the following judgement: l Misc.
Petitions Nos.
401/and 4012 of 1968.
Applications by respondent No. 1 for revocation of the certificate granted by the Madras High Court on 15th October, 1968 under article 133(1)(a) & (b) of the Constitution of India.
AND Civil Appeals Nos. 2183 and 2184 of 1968.
Appeals from the judgment and order elated October 9, 1968 of the Madras High Court in O.S.A. No. 25 and 27 of 1968.
M.C. Setalvad.
V.P. Raman, D.N. Mishra, and J.B. Dadachanji, for the appellant.
section Mohan Kumaramangalam.
M.K. Ramamurthi, Shyamala Pappu and Vineet Kumar, for respondent No. 1.
Rameshwar Nath and Mahinder Narain, for respondent No. 2.
The Judgment of the Court was delivered by Shah, J.
M/s. Tarapore & Company hereinafter called the plaintiffs applied in Suit No. 118 of 1967 for an interim injunction restraining the Bank of India Ltd. the 